In [42]:
# imports
import os
import sys

# Get the directory of the current file
current_dir = os.path.dirname(os.path.realpath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
# Add the current directory and parent directory to the system path
sys.path.insert(0, current_dir)
sys.path.insert(0, parent_dir)

import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
# from pixel_setter import play_video
# from scipy.ndimage import uniform_filter
import importlib
from EMA_functions import *
from DIC_functions import *
import glob
import ast
from sdypy import EMA
import pickle as pkl
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib.gridspec as gridspec
# import re

# Import test data
df_file_description = pd.read_csv('I:/My Drive/PHD/HSC/file_descriptions_wEMA.csv')
df_file_description = df_file_description.loc[:, ~df_file_description.columns.str.startswith('Unnamed')]
# Back up the data
df_file_description.to_csv('I:/My Drive/PHD/HSC/file_descriptions_wEMA_backup.csv')

# List all files
files = glob.glob('D:/thijsmas/HSC/**/*.cihx', recursive=True)

D_in = 40
D_out = 100
nut_wh = (15, 60)
required_parameters = [ 'id0_cam',
                        'id0_for',
                        'double_tap',
                        'smooth_lim',
                        'max_drift',
                        'max_end_drift', 
                        'peak_n', 
                        'peak_F', 
                        'peak_F_threshold', 
                        'shift',
                        'test_number', 
                        'd_lim', 
                        'prey_ij_d', 
                        'prey_ij', 
                        'spider_ij_d', 
                        'spider_ij']

df_file_description.columns

Index(['filename', 'path', 'ecc', 'loc', 'mass', 'taut/loose', 'impact/pluck',
       'version', 'n_frames', 'fps', 'Oversatured pixels',
       'Undersatured pixels', 'prey_ij', 'spider_ij', 'peak_n', 'nut_idx',
       'd_lim', 'smooth_lim', 'max_drift', 'max_end_drift', 'lower', 'upper',
       'pol_order_high', 'added_damping', 'near_zero_threshold',
       'approximate_height', 'peak_F', 'peak_F_threshold', 'shift',
       'test_number', 'higher', 'spider_ij_d', 'prey_ij_d', 'id0_cam',
       'id0_for', 'double_tap', 'root', 'path2', 'max_t_diff', 'max_d_diff'],
      dtype='object')

In [43]:
ecc = 0
loc = 7
version = 0
taut_loose = 'Loose'
impact_pluck = 'Impact'
mass = True
df_filtered = df_file_description[(df_file_description['ecc'] == ecc) & (df_file_description['taut/loose'] == taut_loose) & (df_file_description['impact/pluck'] == impact_pluck) & (df_file_description['mass'] == mass) & (df_file_description['loc'] == loc) & (df_file_description['version'] == version)]
df_filtered

,filename,path,ecc,loc,mass,taut/loose,impact/pluck,version,n_frames,fps,...,higher,spider_ij_d,prey_ij_d,id0_cam,id0_for,double_tap,root,path2,max_t_diff,max_d_diff
190,Full_web_ecc0_new_Floc7_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0.0,7.0,True,Loose,Impact,0.0,10000.0,8000.0,...,300.0,"(517.8862365591398, 286.82779354838704)","(448.29827956989254, 336.15596559139783)",164.0,41801.0,False,D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Fl...,D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Fl...,0.168277,0.405174


In [44]:
name_video = df_filtered['filename'].values[0]
path_video = df_filtered['path2'].values[0]
root_video = df_filtered['root'].values[0]

file_parameters, df_index = unpack_dataframe(df_file_description, name_video, required_parameters)
EMA_structure = EMA_Structure(name_video)
EMA_structure.set_params(**file_parameters)
EMA_structure.set_params(FN0 = 41.2, reaction_time = 0.1)
EMA_structure.open_impact_data()
video = EMA_structure.open_video(add_extension = False)
fps = video.info['Record Rate(fps)']
dt = 1/fps
                 
DIC_structure = DIC_Structure(path_video)

EMA_structure.tp, EMA_structure.d = DIC_structure.join_results([EMA_structure.test_number])
td = EMA_structure.d +  EMA_structure.tp.reshape(len(EMA_structure.tp),1,2)

EMA_structure.initialize_signals()
EMA_structure.initialize_displacement(idx='all', dir='xy')
EMA_structure.nut_idx((EMA_structure.prey_ij[0] + EMA_structure.shift[0], EMA_structure.prey_ij[1] + EMA_structure.shift[1]), exclude_high_amplitude = True, d_lim = EMA_structure.d_lim)

EMA_structure.process_signals(EMA_structure.id0_cam, EMA_structure.id0_for)

EMA_structure.set_freq_properties(padding_ratio=1)
EMA_structure.get_transfer_function(direction='y')

smooth_signals = np.max(np.abs(np.diff(np.linalg.norm(EMA_structure.d, axis=2))), axis = 1) < EMA_structure.smooth_lim
non_drifting = np.abs(np.mean(np.linalg.norm(EMA_structure.d[:,:-100], axis=2), axis=1) < EMA_structure.max_drift)
non_drifting2 = np.abs(np.linalg.norm(EMA_structure.d[:,-1], axis=1)) < EMA_structure.max_end_drift
EMA_structure.valid_tps = smooth_signals & non_drifting & non_drifting2 & EMA_structure.exclude_high_amplitude

lower = 3
higher = 60
pol_order_high = 50


# cam = EMA.Model(EMA_structure.H1[EMA_structure.valid_tps], EMA_structure.freq_camera, lower=lower, upper=higher, pol_order_high=pol_order_high, frf_type = 'receptance')#, get_partfactors=True, dt=dt)
# cam.get_poles(show_progress=True)


D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc7_v0_S01\Full_web_ecc0_new_Floc7_v0_S01.cihx
Available filter methods: eig0, eig1, harris, trigs, harmonic_mean, eig_theta, eig_theta_off, eig0_test, eig1_test


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:104: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


In [39]:
cam = EMA.Model(EMA_structure.H1[EMA_structure.valid_tps], EMA_structure.freq_camera, lower=lower, upper=higher, pol_order_high=pol_order_high, frf_type = 'receptance', get_partfactors=True)#)
cam.get_poles(show_progress=True)
cam.select_poles()
file_path_cam = os.path.join(root_video, f'{name_video}_cam_temp.pkl')
with open(file_path_cam, 'wb') as f:
    pkl.dump(cam, f)

100%|███████████████████████████████████████████████████████████████| 50/50 [00:27<00:00,  1.83it/s]


None


100%|██████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 178.53it/s]
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\matplotlib\text.py:898: ComplexWarning: Casting complex values to real discards the imaginary part
  y = float(self.convert_yunits(self._y))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:888: RuntimeWarning: divide by zero encountered in divide
  p1L = np.kron(np.array([1, 0]), -1/omega**2)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\shape_base.py:1179: RuntimeWarning: invalid value encountered in multiply
  result = _nx.multiply(a_arr, b_arr, subok=(not is_any_mat))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:889: RuntimeWarning: divide by zero encountered in divide
  p2L = np.kron(np.array([0, 1]), -1/omega**2)


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\matplotlib\text.py:757: ComplexWarning: Casting complex values to real discards the imaginary part
  posy = float(self.convert_yunits(self._y))


In [4]:
file_path_cam = os.path.join(root_video, f'{name_video}_cam_temp.pkl')
with open(file_path_cam, 'rb') as file:
    cam = pkl.load(file)

In [40]:
n_modes = 8
participation_factors = cam.partfactors
part_real = []
part_abs = []
for i in range(len(cam.nat_freq)):
    # fn = cam.pole_freq[cam.pole_ind[i][0]][cam.pole_ind[i][1]]
    part_real.append(np.abs(np.real(cam.partfactors[cam.pole_ind[i][0]][cam.pole_ind[i][1]])))
    part_abs.append(np.abs(cam.partfactors[cam.pole_ind[i][0]][cam.pole_ind[i][1]]))

order_real = np.argsort(part_real)[::-1]
order_abs = np.argsort(part_abs)[::-1]

mode_selection = sorted(order_abs[:n_modes])

order_real, order_abs
mean_frf = np.mean(np.abs(cam.frf), axis=0)
fig, ax = plt.subplots(figsize=(10,5))
ax.semilogy(cam.freq,mean_frf, '-')
# ax.vlines(np.array(cam.nat_freq)[order_real[:n_modes]], ymin = 1e0, ymax = 1e4, color='r', linestyle='-')
ax.vlines(np.array(cam.nat_freq)[order_abs[:n_modes]], ymin = 1e0, ymax = 1e4, color='g', linestyle='--')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('FRF (receptance) [pixel/N]')


Text(0, 0.5, 'FRF (receptance) [pixel/N]')

In [45]:
%matplotlib qt
name_video_base = name_video.split('.')[0]
file_name_dist = os.path.join(root_video, f"{name_video_base}_dist.pkl")
with open(file_name_dist, 'rb') as f:
    dist = pkl.load(f)
homography_matrix = dist["homography_matrix"]
video = EMA_structure.open_video(add_extension = False)
still_image = video.reader.get_frame(0)
height, width = still_image.shape
undistorted_image = cv2.warpPerspective(still_image, homography_matrix, (width, height))
zero_columns = np.all(undistorted_image == 0, axis=0)
first_true = np.argmax(~zero_columns)  # First True
last_true = len(zero_columns) - np.argmax(~zero_columns[::-1]) - 1

tp = EMA_structure.tp[EMA_structure.valid_tps]
tp = np.dot(homography_matrix, np.array([tp[:, 1], tp[:, 0], np.ones(tp.shape[0])]))
tp = (tp[1::-1]/tp[2][np.newaxis, :]).T
# tp = tp #- np.array([-height/2, width/2])

fig = plt.figure(figsize=(21, 13)) 
gs = gridspec.GridSpec(3, n_modes, height_ratios=[.6, .6, 1]) #height_ratios=[.5, 1, 1, 1],
ax_real = [fig.add_subplot(gs[0, i]) for i in range(n_modes)]
ax_imag = [fig.add_subplot(gs[1, i]) for i in range(n_modes)]
ax_FRF =  fig.add_subplot(gs[2, :2])
ax_Sxx  = fig.add_subplot(gs[2, 2:4])
ax_MAC  = fig.add_subplot(gs[2, 4:6])
ax_info = fig.add_subplot(gs[2, 6:])


ax_FRF.semilogy(cam.freq, mean_frf, '-')
for mode, mode_f in zip(mode_selection, np.array(cam.nat_freq)[mode_selection]):
    ax_FRF.vlines(mode_f, ymin=1e0, ymax=1e4, color='g', linestyle='--')
    ax_FRF.text(mode_f, 1e4, f"{mode+1:}", ha='center', va='bottom')
ax_FRF.set_xlabel('Frequency [Hz]')
ax_FRF.set_ylabel('FRF (receptance) [pixel/N]')

fig, ax_MAC = plot_MAC(cam, fig, ax_MAC, mode_vec = mode_selection)

Acquisition_period  = EMA_structure.t_camera[-1]
S_xx = 1/Acquisition_period * np.conj(EMA_structure.disp_fft_y) * EMA_structure.disp_fft_y
mean_S_xx = np.mean(S_xx[EMA_structure.valid_tps], axis=0)
arg_higer = np.argmin(np.abs(EMA_structure.freq_camera - higher))
S_xx_min, S_xx_max = np.min(mean_S_xx[:arg_higer]), np.max(mean_S_xx[:arg_higer])
ax_Sxx.semilogy(EMA_structure.freq_camera, mean_S_xx, '-', label='raw')
ax_Sxx.set_xlim([0, higher])
ax_Sxx.set_ylim([S_xx_min*0.8, S_xx_max*1.3])
ax_Sxx.set_title('S_xx [pixel^2]')
ax_Sxx.set_xlabel('Frequency [Hz]')
for mode, mode_f in zip(mode_selection, np.array(cam.nat_freq)[mode_selection]):
    ax_Sxx.vlines(mode_f, ymin=S_xx_min, ymax=S_xx_max, color='g', linestyle='--')  
    ax_Sxx.text(mode_f, S_xx_max, f"{mode+1:}", ha='center', va='bottom')

# EMA_structure.set_freq_properties(padding_ratio=2)
# S_xx = 1/Acquisition_period * np.conj(EMA_structure.disp_fft_y) * EMA_structure.disp_fft_y
# mean_S_xx = np.mean(S_xx[EMA_structure.valid_tps], axis=0)
# arg_higer = np.argmin(np.abs(EMA_structure.freq_camera - higher))
# S_xx_min, S_xx_max = np.min(mean_S_xx[:arg_higer]), np.max(mean_S_xx[:arg_higer])
# ax_Sxx.semilogy(EMA_structure.freq_camera, mean_S_xx, '--', label = 'padded', alpha=0.5)
nat_freq = np.array(cam.nat_freq)[mode_selection]
nat_freq_arg = np.argmin(np.abs(EMA_structure.freq_camera[:, np.newaxis] - nat_freq), axis=0)
Sxx_peaks = np.abs(mean_S_xx[nat_freq_arg])
Sxx_peaks_norm = Sxx_peaks/Sxx_peaks[0]
ax_Sxx.scatter(nat_freq, Sxx_peaks, color='m', s=40, marker='x')

Sxx_peaks_norm_str = '\n'.join([f"mode {mode+1}: {peak:.2g}" for mode, peak in zip(mode_selection, Sxx_peaks_norm)])
info_text0 = f"""
            Figure Information:
            - Peak Force: {EMA_structure.peak_F:.2f} N
            - double tap: {EMA_structure.double_tap}
            - Prey distance: {np.linalg.norm([EMA_structure.prey_ij_d[0]-EMA_structure.spider_ij_d[0], EMA_structure.prey_ij_d[1] - EMA_structure.spider_ij_d[1]]):.2f} px
            - peak S_xx relative to first peak:\n {Sxx_peaks_norm_str}
            """

ax_info.text(0.5, 1, info_text0, transform=ax_info.transAxes, fontsize=12, verticalalignment='top', horizontalalignment='center')
ax_info.axis('off')
fig.suptitle(EMA_structure.file_name)

for ax_real, ax_imag, mode_n in zip(ax_real, ax_imag, sorted(order_abs[:n_modes])):
    ax_real.imshow(undistorted_image, cmap = 'gray')
    ax_real.axis('off')
    ax_real.set_xlim([first_true, last_true])
    ax_imag.imshow(undistorted_image, cmap = 'gray')
    ax_imag.set_xlim([first_true, last_true])
    ax_imag.axis('off')
    fig, ax_real, ax_imag = plot_mode_shape_flat(cam, fig, ax_real, ax_imag, mode_n, tp)
    ax_real.scatter([EMA_structure.spider_ij_d[0], EMA_structure.prey_ij_d[0]], [EMA_structure.spider_ij_d[1], EMA_structure.prey_ij_d[1]], color='g', s=20, marker='x')
    ax_imag.scatter([EMA_structure.spider_ij_d[0], EMA_structure.prey_ij_d[0]], [EMA_structure.spider_ij_d[1], EMA_structure.prey_ij_d[1]], color='g', s=30, marker='x')
mode_selection, Sxx_peaks

D:/thijsmas/HSC\ecc0\web0\Full_web_ecc0_new_Floc7_v0_S01\Full_web_ecc0_new_Floc7_v0_S01.cihx


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\matplotlib\cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\matplotlib\cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\matplotlib\transforms.py:993: ComplexWarning: Casting complex values to real discards the imaginary part
  self._points[:, 1] = interval
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\numpy\ma\core.py:3375: ComplexWarning: Casting complex values to real discards the imaginary part
  _data[indx] = dval
C:\Users\thijsmas\Documents\GitHub\pyidi\scripts\EMA_functio

([0, 1, 2, 3, 5, 6, 12, 13],
 array([1.23705808e+00, 1.52247481e-02, 9.91851251e-01, 4.22513830e-03,
        2.74572646e-03, 6.50884822e-03, 7.36312650e-04, 2.87712589e-04]))

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\matplotlib\text.py:757: ComplexWarning: Casting complex values to real discards the imaginary part
  posy = float(self.convert_yunits(self._y))


In [ ]:
file_path_cam = os.path.join(root_video, f'{name_video}_cam.pkl')
with open(file_path_cam, 'rb') as file:
    cam_saved = pkl.load(file)

In [ ]:
for file_i, (name_video, path_video, root_video) in enumerate(zip(df_filtered['filename'], df_filtered['path2'], df_filtered['root'])):
    if pd.isna(path_video):
        print(f'Could not open {name_video}')
        continue

    
    file_parameters, df_index = unpack_dataframe(df_file_description, name_video, required_parameters)
    if file_parameters is None:
        print(f'some items in {name_video} could not be unpacked')
        continue

    EMA_structure = EMA_Structure(name_video)
    EMA_structure.set_params(**file_parameters)
    EMA_structure.set_params(FN0 = 41.2, reaction_time = 0.1) # FN0 from Lott: Prey localization in spider orb webs using modal vibration analysis. reaction_time from (Klärner and Barth1982)
    
    # Open impact data
    EMA_structure.open_impact_data()
    video = EMA_structure.open_video(add_extension = False)
    try:
        comment = EMA_structure.impact_data['comment']
        print(f"Comment: {comment}")
    except:
        print(f'File {name_video} not found in impact data')

        continue

    # Open displacement data
    DIC_structure = DIC_Structure(path_video)
    # video = DIC_structure.video
    df = DIC_structure.list_test_data(test_range = range(1, 100), robostness_check = False)
    print(df)
    try:
        last_row = df.iloc[-1]  # Get the last row of the DataFrame
    except:
        print(f'File {name_video} not found in DIC data')
        continue
    
    file_path_cam = os.path.join(root_video, f'{name_video}_cam.pkl')
    if os.path.exists(file_path_cam):
        cam = pkl.load(open(file_path_cam, 'rb'))
        normalized = True
        try:
            if cam.nat_freq[0]==0:
                EMA_structure.fn0 = cam.nat_freq[1]
            else:
                EMA_structure.fn0 = cam.nat_freq[0]
        except:
            normalized = False
            EMA_structure.fn0 = 0
    else:
        normalized = False
        EMA_structure.fn0 = 0

    if not normalized:
        print(f'File {name_video} cannot be normalized')
        continue